In [1]:
driver.close()

NameError: name 'driver' is not defined

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize Firefox Options
firefox_options = FirefoxOptions()

# Create a new Firefox Profile for your extensions
firefox_profile = webdriver.FirefoxProfile()

# Convert the profile to a base64 string and add it to the options
firefox_options.profile = firefox_profile

In [2]:
#Webscrpaing and request parsing 
import requests
from bs4 import BeautifulSoup, Comment
import time

from lxml import etree
from lxml.html import fromstring

In [3]:
from googlesearch import search

In [4]:
import pandas as pd
import re

/var/folders/jq/vkbby4bd33xbf6kks2_zrswh0000gn/T/ipykernel_22961/581249992.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
# Set up the service with GeckoDriverManager
service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=firefox_options)

driver.install_addon('/Users/allensunny/Downloads/Mozilla Scraper Tools/uBlock0_1.55.0.firefox.signed.xpi',  temporary=True)
driver.install_addon('/Users/allensunny/Downloads/Mozilla Scraper Tools/bypass_paywalls_clean-3.5.5.0.xpi', temporary=True)

'magnolia@12.34'

In [6]:
#Proxy set up
#Consutructing the proxy farm 
with open('/Users/allensunny/Downloads/http.txt', 'r') as file:
    content = file.read()

#Converting to a list of proxies to fetch data
proxies = content.strip().split('\n')

In [7]:
#Proxy Function extraction 
def get_random_proxy():
    return random.choice(proxies)

In [ ]:
#moving to the next page
def click_next():
    try:
        # Wait for the "Next" button to be clickable
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "li.next a"))
        )
        # Click the "Next" button if it's found and clickable
        next_button.click()
        return True
    except (TimeoutException, NoSuchElementException):
        # Return False if the "Next" button isn't present or clickable
        return False

In [8]:
#Extracting from the wsj 
url = "https://www.wsj.com/market-data/quotes/company-list"

In [9]:
# Navigate to the URL
driver.get(url)

# Wait for the h3 tag
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h3")))

# Pass the page source to BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the h3 tag with 'Browse by Country'
h3 = soup.find('h3', string=re.compile('Browse by Country', re.IGNORECASE))

# Initialize an empty list to hold all URLs and country names
country_data = []

# If the h3 tag is found, find the related elements
if h3:
    next_ul = h3.find_next_sibling('ul')
    if next_ul:
        li_elements = next_ul.find_all('li')
        for li in li_elements:
            a_tag = li.find('a')
            if a_tag:
                # Store each URL and the country name in the list
                country_data.append((a_tag['href'], a_tag.text))

# Iterate through the list of URLs, navigate to each one, and extract table data
Main_df = pd.DataFrame()
for url, country_name in country_data:
    driver.get(url)
     
    #Initalize the country wise dataframe
    Country_df = pd.DataFrame()
    while True:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract table data...
        table = soup.find('table')
        if table:
            headers = [header.get_text(strip=True) for header in table.find_all('th')]
            rows = []
            for row in table.find_all('tr'):
                cells = row.find_all('td')
                if cells:
                    rows.append([cell.get_text(strip=True) for cell in cells])

            # Create a temporary DataFrame and append it to the main DataFrame
        temp_df = pd.DataFrame(rows, columns=headers)
        temp_df['Country'] = country_name  # Add the country name to each row

        # Concatenate the temporary DataFrame with the main DataFrame
        Country_df = pd.concat([Country_df, temp_df], ignore_index=True)
        
        # Check if the 'Next' button exists
        next_button = driver.find_elements(By.CSS_SELECTOR, "li.next a")
        if not next_button:
            break  # No 'Next' button, exit the loop and proceed to the next URL

        try:
            # Try clicking the 'Next' button
            next_button[0].click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button[0]))
        except Exception as e:
            #print(f"Cannot click 'Next' button, moving to the next URL. Exception: {e}")
            break

    #Append to the main df
    Main_df = pd.concat([Main_df, Country_df], ignore_index=True)

# Close the driver
driver.quit()

In [10]:
Main_df

,Name,Exchange,SuperSector,Country
0,01 Communique Laboratory Inc.(OONEF),OOTC,Software,United States
1,10X Capital Venture Acquisition Corp. II Cl A(...,XNAS,Shell companies,United States
2,10X Capital Venture Acquisition Corp. II Un(VC...,XNAS,Shell companies,United States
3,10X Capital Venture Acquisition Corp. II Wt(VC...,XNAS,,United States
4,10X Capital Venture Acquisition Corp. III(VCXB...,XNYS,Shell companies,United States
...,...,...,...,...
55158,Yukselen Celik A.S.(YKSLN.E),XIST,Iron/Steel,Turkey
55159,Yunlu Sanayi ve Ticaret A.S.(YUNSA.E),XIST,Clothing,Turkey
55160,Zedur Enerji Elektrik Uretim A.S.(ZEDUR.E),XIST,Construction,Turkey
55161,Ziraat Gayrimenkul Yatirim Ortakligi A.S.(ZRGY...,XIST,Industrial/Office REITs,Turkey


In [15]:
Main_df.to_csv('Companies_list.csv', index=False)